In [14]:
#%pip install beautifulsoup4

# Não Presta

In [8]:
import requests
from bs4 import BeautifulSoup

def get_acoes_sem_prejuizo():
    url = "https://www.oceans14.com.br/acoes/semPrejuizo"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.google.com/"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # Localiza o bloco principal com os tickers
    bloco_tickers = soup.find("div", id="ctl00_sldTopoCotacoesCorpo")

    tickers = []
    if bloco_tickers:
        for span in bloco_tickers.find_all("span", class_="cotacaoTopoTicker"):
            ticker = span.get_text(strip=True)
            if ticker and len(ticker) <= 6:  # Ticker típico brasileiro
                tickers.append(ticker)

    return tickers

# Teste
tickers = get_acoes_sem_prejuizo()
print("Ações sem prejuízo encontradas:", tickers)


Ações sem prejuízo encontradas: ['SNLG11', 'RCFA11', 'RBDS11', 'RADL3', 'ETER3', 'ANIM3', 'PRSV11', 'RCSL3', 'BEEF3', 'USIM3', 'TOKY3', 'HAAA11', 'KNRE11', 'PCAR3', 'BLAU3', 'CTXT11', 'ARXD11', 'SCPF11', 'LIFE11', 'BPAC3']


In [9]:
get_acoes_sem_prejuizo()

['SNLG11',
 'RCFA11',
 'RBDS11',
 'RADL3',
 'ETER3',
 'ANIM3',
 'PRSV11',
 'RCSL3',
 'BEEF3',
 'USIM3',
 'TOKY3',
 'HAAA11',
 'KNRE11',
 'PCAR3',
 'BLAU3',
 'CTXT11',
 'ARXD11',
 'SCPF11',
 'LIFE11',
 'BPAC3']

# Presta

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import yfinance as yf

def get_detalhes_acao_oceans14(nome, ticker):
    url = f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/balanco-dividendos"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    def get_by_id(id_):
        el = soup.find(id=id_)
        return el.get_text(strip=True) if el else None

    def buscar_variacao_12m():
        el = soup.find(id="ctl00_conteudoPrincipal_lblCotacaoVariacao12mTicker01Desktop")
        if el:
            match = re.search(r"Var\. 12m:\s*(.*)", el.text)
            return match.group(1).strip() if match else None
        return None

    def get_lista_abas():
        abas = soup.select(".nav-tabs > li > span")
        return [aba.get_text(strip=True) for aba in abas if aba.get("data-target")]

    def get_links_graficos():
        links = []
        for aba in ["abaCotacao", "abaCotacaoLucro", "abaHistoricoPL"]:
            if soup.find(id=aba):
                links.append(f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/{aba}")
        return links

    dados = {
        "nome_url": nome.upper(),
        "ticker": ticker.upper(),
        "razao_social": get_by_id("ctl00_conteudoPrincipal_lblRazaoSocial"),
        "nome_fantasia": soup.select_one("span[style*='font-size:16px']").text.strip() if soup.select_one("span[style*='font-size:16px']") else None,
        "cnpj": get_by_id("ctl00_conteudoPrincipal_lblCnpj"),
        "quantidade_acoes": get_by_id("ctl00_conteudoPrincipal_lblNumeroAcoes"),
        "free_float": get_by_id("ctl00_conteudoPrincipal_lblFreeFloat"),
        "quantidade_acionistas": get_by_id("ctl00_conteudoPrincipal_lblQntAcionistas"),
        "valor_mercado": get_by_id("ctl00_conteudoPrincipal_lblValorMercado"),
        "cotacao_atual": get_by_id("ctl00_conteudoPrincipal_lblCotacaoTicker01Desktop"),
        "variacao_dia": soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div").text.strip() if soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div") else None,
        "variacao_12m": buscar_variacao_12m(),
        "data_hora_cotacao": get_by_id("ctl00_conteudoPrincipal_lblCotacaoDataHoraTicker01Desktop"),
        "segmento_listagem": get_by_id("ctl00_conteudoPrincipal_lblGovernanca"),
        "setor": get_by_id("ctl00_conteudoPrincipal_lblSetor"),
        "subsetor": get_by_id("ctl00_conteudoPrincipal_lblSubsetor"),
        "segmento": get_by_id("ctl00_conteudoPrincipal_lblSegmento"),
        "site_ri": get_by_id("ctl00_conteudoPrincipal_lblSiteRI"),
        "ultimo_resultado": get_by_id("ctl00_conteudoPrincipal_lblUltimoBalanco"),
        "proximo_resultado": get_by_id("ctl00_conteudoPrincipal_lblProximoBalanco"),
        "abas_disponiveis": get_lista_abas(),
        "links_graficos": get_links_graficos(),
    }
    
    
    try:
        yf_ticker = yf.Ticker(f"{ticker}.SA")
        info = yf_ticker.info
        dados["pl"] = info.get("trailingPE")
        dados["pvp"] = info.get("priceToBook")
        dados["roe"] = info.get("returnOnEquity")
        dados["roa"] = info.get("returnOnAssets")
        dados["roi"] = info.get("returnOnInvestment")
        dados["dividend_yield"] = info.get("dividendYield")
        dados["eps"] = info.get("trailingEps")
        dados["vpa"] = info.get("bookValue")
        dados["beta"] = info.get("beta")
        dados["debt_to_equity"] = info.get("debtToEquity")
    except Exception as e:
        dados["erro_yahoo_finance"] = str(e)
        dados["historico_financeiro"] = []
    return pd.DataFrame([dados])

def Dados_financeniros_da_empresa(ticker):# Financeiros históricos via Yahoo Finance
    try:
        yf_ticker = yf.Ticker(f"{ticker}.SA")
        info = yf_ticker.info
        income_stmt = yf_ticker.income_stmt
        balance_sheet = yf_ticker.balance_sheet
        cashflow = yf_ticker.cashflow

        historico_financeiro = []

        for col in income_stmt.columns:
            ano = col.year if hasattr(col, "year") else str(col)

            dados_ano = {
                "ano": ano,
                "total_revenue": income_stmt.loc["Total Revenue", col] if "Total Revenue" in income_stmt.index else None,
                "gross_profit": income_stmt.loc["Gross Profit", col] if "Gross Profit" in income_stmt.index else None,
                "net_income": income_stmt.loc["Net Income", col] if "Net Income" in income_stmt.index else None,
                "ebit": income_stmt.loc["EBIT", col] if "EBIT" in income_stmt.index else None,
                "operating_income": income_stmt.loc["Operating Income", col] if "Operating Income" in income_stmt.index else None,
                "interest_expense": income_stmt.loc["Interest Expense", col] if "Interest Expense" in income_stmt.index else None,
                "income_before_tax": income_stmt.loc["Income Before Tax", col] if "Income Before Tax" in income_stmt.index else None,
                "research_and_dev": income_stmt.loc["Research Development", col] if "Research Development" in income_stmt.index else None,
                "total_assets": balance_sheet.loc["Total Assets", col] if "Total Assets" in balance_sheet.index else None,
                "total_liab": balance_sheet.loc["Total Liab", col] if "Total Liab" in balance_sheet.index else None,
                "total_equity": balance_sheet.loc["Total Stockholder Equity", col] if "Total Stockholder Equity" in balance_sheet.index else None,
                "cash": balance_sheet.loc["Cash", col] if "Cash" in balance_sheet.index else None,
                "short_term_debt": balance_sheet.loc["Short Long Term Debt", col] if "Short Long Term Debt" in balance_sheet.index else None,
                "long_term_debt": balance_sheet.loc["Long Term Debt", col] if "Long Term Debt" in balance_sheet.index else None,
                "free_cash_flow": cashflow.loc["Free Cash Flow", col] if "Free Cash Flow" in cashflow.index else None,
                "capital_expenditures": cashflow.loc["Capital Expenditures", col] if "Capital Expenditures" in cashflow.index else None,
                "cash_from_operations": cashflow.loc["Total Cash From Operating Activities", col] if "Total Cash From Operating Activities" in cashflow.index else None,
            }

            if dados_ano["gross_profit"] and dados_ano["total_revenue"]:
                try:
                    dados_ano["margem_bruta_percentual"] = round((dados_ano["gross_profit"] / dados_ano["total_revenue"]) * 100, 2)
                except:
                    dados_ano["margem_bruta_percentual"] = None
            else:
                dados_ano["margem_bruta_percentual"] = None

            historico_financeiro.append(dados_ano)
    except Exception as e:
        historico_financeiro["erro_yahoo_finance"] = str(e)
        historico_financeiro["historico_financeiro"] = []
    return pd.DataFrame(historico_financeiro)


if __name__ == "__main__":
    empresa = get_detalhes_acao_oceans14("weg","WEGE3" )
    pd.set_option("display.max_rows", None)
    dados_historicos=Dados_financeniros_da_empresa("WEGE3")
    print(empresa.T)


                                                                       0
nome_url                                                             WEG
ticker                                                             WEGE3
razao_social                                                     WEG S.A
nome_fantasia                                                        WEG
cnpj                                                  84.429.695/0001-11
quantidade_acoes                                           4.197.318.000
free_float                                                   35,33% (ON)
quantidade_acionistas  334.172 (PFs), 1.607 (PJs) e 1.201 (Institucio...
valor_mercado                                           R$ 156,1 bilhões
cotacao_atual                                                   R$ 39,02
variacao_dia                                                       2,25%
variacao_12m                                                     -17,90%
data_hora_cotacao                                  

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import yfinance as yf

def get_detalhes_acao_oceans14(nome, ticker):
    url = f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/balanco-dividendos"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    def get_by_id(id_):
        el = soup.find(id=id_)
        return el.get_text(strip=True) if el else None

    def buscar_variacao_12m():
        el = soup.find(id="ctl00_conteudoPrincipal_lblCotacaoVariacao12mTicker01Desktop")
        if el:
            match = re.search(r"Var\. 12m:\s*(.*)", el.text)
            return match.group(1).strip() if match else None
        return None

    def get_lista_abas():
        abas = soup.select(".nav-tabs > li > span")
        return [aba.get_text(strip=True) for aba in abas if aba.get("data-target")]

    def get_links_graficos():
        links = []
        for aba in ["abaCotacao", "abaCotacaoLucro", "abaHistoricoPL"]:
            if soup.find(id=aba):
                links.append(f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/{aba}")
        return links

    dados = {
        "nome_url": nome.upper(),
        "ticker": ticker.upper(),
        "razao_social": get_by_id("ctl00_conteudoPrincipal_lblRazaoSocial"),
        "nome_fantasia": soup.select_one("span[style*='font-size:16px']").text.strip() if soup.select_one("span[style*='font-size:16px']") else None,
        "cnpj": get_by_id("ctl00_conteudoPrincipal_lblCnpj"),
        "quantidade_acoes": get_by_id("ctl00_conteudoPrincipal_lblNumeroAcoes"),
        "free_float": get_by_id("ctl00_conteudoPrincipal_lblFreeFloat"),
        "quantidade_acionistas": get_by_id("ctl00_conteudoPrincipal_lblQntAcionistas"),
        "valor_mercado": get_by_id("ctl00_conteudoPrincipal_lblValorMercado"),
        "cotacao_atual": get_by_id("ctl00_conteudoPrincipal_lblCotacaoTicker01Desktop"),
        "variacao_dia": soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div").text.strip() if soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div") else None,
        "variacao_12m": buscar_variacao_12m(),
        "data_hora_cotacao": get_by_id("ctl00_conteudoPrincipal_lblCotacaoDataHoraTicker01Desktop"),
        "segmento_listagem": get_by_id("ctl00_conteudoPrincipal_lblGovernanca"),
        "setor": get_by_id("ctl00_conteudoPrincipal_lblSetor"),
        "subsetor": get_by_id("ctl00_conteudoPrincipal_lblSubsetor"),
        "segmento": get_by_id("ctl00_conteudoPrincipal_lblSegmento"),
        "site_ri": get_by_id("ctl00_conteudoPrincipal_lblSiteRI"),
        "ultimo_resultado": get_by_id("ctl00_conteudoPrincipal_lblUltimoBalanco"),
        "proximo_resultado": get_by_id("ctl00_conteudoPrincipal_lblProximoBalanco"),
        "abas_disponiveis": get_lista_abas(),
        "links_graficos": get_links_graficos(),
    }

    # Financeiros históricos via Yahoo Finance
    try:
        yf_ticker = yf.Ticker(f"{ticker}.SA")
        info = yf_ticker.info
        income_stmt = yf_ticker.income_stmt
        balance_sheet = yf_ticker.balance_sheet
        cashflow = yf_ticker.cashflow

        historico_financeiro = []

        for col in income_stmt.columns:
            ano = col.year if hasattr(col, "year") else str(col)

            dados_ano = {
                "ano": ano,
                "total_revenue": income_stmt.loc["Total Revenue", col] if "Total Revenue" in income_stmt.index else None,
                "gross_profit": income_stmt.loc["Gross Profit", col] if "Gross Profit" in income_stmt.index else None,
                "net_income": income_stmt.loc["Net Income", col] if "Net Income" in income_stmt.index else None,
                "ebit": income_stmt.loc["EBIT", col] if "EBIT" in income_stmt.index else None,
                "operating_income": income_stmt.loc["Operating Income", col] if "Operating Income" in income_stmt.index else None,
                "interest_expense": income_stmt.loc["Interest Expense", col] if "Interest Expense" in income_stmt.index else None,
                "income_before_tax": income_stmt.loc["Income Before Tax", col] if "Income Before Tax" in income_stmt.index else None,
                "research_and_dev": income_stmt.loc["Research Development", col] if "Research Development" in income_stmt.index else None,
                "total_assets": balance_sheet.loc["Total Assets", col] if "Total Assets" in balance_sheet.index else None,
                "total_liab": balance_sheet.loc["Total Liab", col] if "Total Liab" in balance_sheet.index else None,
                "total_equity": balance_sheet.loc["Total Stockholder Equity", col] if "Total Stockholder Equity" in balance_sheet.index else None,
                "cash": balance_sheet.loc["Cash", col] if "Cash" in balance_sheet.index else None,
                "short_term_debt": balance_sheet.loc["Short Long Term Debt", col] if "Short Long Term Debt" in balance_sheet.index else None,
                "long_term_debt": balance_sheet.loc["Long Term Debt", col] if "Long Term Debt" in balance_sheet.index else None,
                "free_cash_flow": cashflow.loc["Free Cash Flow", col] if "Free Cash Flow" in cashflow.index else None,
                "capital_expenditures": cashflow.loc["Capital Expenditures", col] if "Capital Expenditures" in cashflow.index else None,
                "cash_from_operations": cashflow.loc["Total Cash From Operating Activities", col] if "Total Cash From Operating Activities" in cashflow.index else None,
            }

            if dados_ano["gross_profit"] and dados_ano["total_revenue"]:
                try:
                    dados_ano["margem_bruta_percentual"] = round((dados_ano["gross_profit"] / dados_ano["total_revenue"]) * 100, 2)
                except:
                    dados_ano["margem_bruta_percentual"] = None
            else:
                dados_ano["margem_bruta_percentual"] = None

            historico_financeiro.append(dados_ano)

        dados["historico_financeiro"] = historico_financeiro

        dados["pl"] = info.get("trailingPE")
        dados["pvp"] = info.get("priceToBook")
        dados["roe"] = info.get("returnOnEquity")
        dados["roa"] = info.get("returnOnAssets")
        dados["roi"] = info.get("returnOnInvestment")
        dados["dividend_yield"] = info.get("dividendYield")
        dados["eps"] = info.get("trailingEps")
        dados["vpa"] = info.get("bookValue")
        dados["beta"] = info.get("beta")
        dados["debt_to_equity"] = info.get("debtToEquity")

    except Exception as e:
        dados["erro_yahoo_finance"] = str(e)
        dados["historico_financeiro"] = []

    return pd.DataFrame([dados])

if __name__ == "__main__":
    teste = get_detalhes_acao_oceans14("weg", "WEGE3")
    pd.set_option("display.max_rows", None)
    print(teste.T)

                                                                       0
nome_url                                                             WEG
ticker                                                             WEGE3
razao_social                                                     WEG S.A
nome_fantasia                                                        WEG
cnpj                                                  84.429.695/0001-11
quantidade_acoes                                           4.197.318.000
free_float                                                   35,33% (ON)
quantidade_acionistas  334.172 (PFs), 1.607 (PJs) e 1.201 (Institucio...
valor_mercado                                           R$ 156,1 bilhões
cotacao_atual                                                   R$ 39,02
variacao_dia                                                       2,25%
variacao_12m                                                     -17,90%
data_hora_cotacao                                  

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


def get_detalhes_acao_oceans14(nome, ticker):
    url = f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/balanco-dividendos"
    headers = {"User-Agent": "Mozilla/5.0"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    def get_by_id(id_):
        el = soup.find(id=id_)
        return el.get_text(strip=True) if el else None

    def buscar_variacao_12m():
        el = soup.find(id="ctl00_conteudoPrincipal_lblCotacaoVariacao12mTicker01Desktop")
        if el:
            match = re.search(r"Var\. 12m:\s*(.*)", el.text)
            return match.group(1).strip() if match else None
        return None

    def get_lista_abas():
        abas = soup.select(".nav-tabs > li > span")
        return [aba.get_text(strip=True) for aba in abas if aba.get("data-target")]

    def get_links_graficos():
        links = []
        for aba in ["abaCotacao", "abaCotacaoLucro", "abaHistoricoPL"]:
            if soup.find(id=aba):
                links.append(f"https://www.oceans14.com.br/acoes/{nome.lower()}/{ticker.lower()}/{aba}")
        return links

    def extrair_valor_por_titulo(titulo):
        # Procura o título nas divs do painel financeiro
        linhas = soup.select("#abaResultados .row")
        for linha in linhas:
            colunas = linha.find_all("div")
            if len(colunas) >= 2:
                nome = colunas[0].text.strip().lower()
                valor = colunas[1].text.strip()
                if titulo.lower() in nome:
                    return valor
        return None

    # Extração básica
    dados = {
        "nome_url": nome.upper(),
        "ticker": ticker.upper(),
        "razao_social": get_by_id("ctl00_conteudoPrincipal_lblRazaoSocial"),
        "nome_fantasia": soup.select_one("span[style*='font-size:16px']").text.strip() if soup.select_one("span[style*='font-size:16px']") else None,
        "cnpj": get_by_id("ctl00_conteudoPrincipal_lblCnpj"),
        "quantidade_acoes": get_by_id("ctl00_conteudoPrincipal_lblNumeroAcoes"),
        "free_float": get_by_id("ctl00_conteudoPrincipal_lblFreeFloat"),
        "quantidade_acionistas": get_by_id("ctl00_conteudoPrincipal_lblQntAcionistas"),
        "valor_mercado": get_by_id("ctl00_conteudoPrincipal_lblValorMercado"),
        "cotacao_atual": get_by_id("ctl00_conteudoPrincipal_lblCotacaoTicker01Desktop"),
        "variacao_dia": soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div").text.strip() if soup.select_one("#ctl00_conteudoPrincipal_lblCotacaoVariacao1dTicker01Desktop div") else None,
        "variacao_12m": buscar_variacao_12m(),
        "data_hora_cotacao": get_by_id("ctl00_conteudoPrincipal_lblCotacaoDataHoraTicker01Desktop"),
        "segmento_listagem": get_by_id("ctl00_conteudoPrincipal_lblGovernanca"),
        "setor": get_by_id("ctl00_conteudoPrincipal_lblSetor"),
        "subsetor": get_by_id("ctl00_conteudoPrincipal_lblSubsetor"),
        "segmento": get_by_id("ctl00_conteudoPrincipal_lblSegmento"),
        "site_ri": get_by_id("ctl00_conteudoPrincipal_lblSiteRI"),
        "ultimo_resultado": get_by_id("ctl00_conteudoPrincipal_lblUltimoBalanco"),
        "proximo_resultado": get_by_id("ctl00_conteudoPrincipal_lblProximoBalanco"),
        "abas_disponiveis": get_lista_abas(),
        "links_graficos": get_links_graficos(),
    }

    # Financeiros (demonstrativo)
    receita_liquida = extrair_valor_por_titulo("receita líquida")
    lucro_bruto = extrair_valor_por_titulo("lucro bruto")
    lucro_liquido = extrair_valor_por_titulo("lucro líquido")
    ebitda = extrair_valor_por_titulo("ebitda")
    ebit = extrair_valor_por_titulo("ebit")

    dados["receita_liquida"] = receita_liquida
    dados["lucro_bruto"] = lucro_bruto
    dados["lucro_liquido"] = lucro_liquido
    dados["ebitda"] = ebitda
    dados["ebit"] = ebit

    # Cálculo de margem bruta se possível
    try:
        receita_valor = float(receita_liquida.replace(".", "").replace(",", ".").replace("R$", "").replace("mil", "").strip())
        lucro_valor = float(lucro_bruto.replace(".", "").replace(",", ".").replace("R$", "").replace("mil", "").strip())
        margem_bruta = (lucro_valor / receita_valor) * 100 if receita_valor != 0 else None
        dados["margem_bruta_percentual"] = f"{margem_bruta:.2f}%"
    except:
        dados["margem_bruta_percentual"] = None

    return pd.DataFrame([dados])


# Exemplo de uso
if __name__ == "__main__":
    df = get_detalhes_acao_oceans14("weg", "WEGE3")
    pd.set_option("display.max_rows", None)
    print(df.T)


                                                                         0
nome_url                                                               WEG
ticker                                                               WEGE3
razao_social                                                       WEG S.A
nome_fantasia                                                          WEG
cnpj                                                    84.429.695/0001-11
quantidade_acoes                                             4.197.318.000
free_float                                                     35,33% (ON)
quantidade_acionistas    334.172 (PFs), 1.607 (PJs) e 1.201 (Institucio...
valor_mercado                                             R$ 156,1 bilhões
cotacao_atual                                                     R$ 38,02
variacao_dia                                                         2,40%
variacao_12m                                                       -21,07%
data_hora_cotacao        

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_acoes_sem_prejuizo():
    url = "https://www.oceans14.com.br/acoes/semPrejuizo"
    headers = {"User-Agent": "Mozilla/5.0"}

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    tabela = soup.find("table", id="t1")
    rows = tabela.find("tbody").find_all("tr")
    lista = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 5:
            link = cols[0].find("a")
            ticker = link.get_text(strip=True) if link else None
            lista.append(ticker)
    return lista
        
def get_acoes_sem_prejuizo_detalhado():
    url = "https://www.oceans14.com.br/acoes/semPrejuizo"
    headers = {"User-Agent": "Mozilla/5.0"}

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    tabela = soup.find("table", id="t1")
    rows = tabela.find("tbody").find_all("tr")
    lista = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 5:
            link = cols[0].find("a")
            ticker = link.get_text(strip=True) if link else None
            empresa = cols[1].get_text(strip=True)
            segmento = cols[2].get_text(strip=True)
            valor_mercado = cols[3].get_text(strip=True)
            lucro_12m = cols[4].get_text(strip=True)
            lista.append({
                "ticker": ticker,
                "empresa": empresa,
                "segmento": segmento,
                "valor_mercado": valor_mercado,
                "lucro_12m": lucro_12m
            })
    return pd.DataFrame(lista)

if __name__ == "__main__":
    df = get_acoes_sem_prejuizo_detalhado()
    print(df)


   ticker             empresa                                  segmento  \
0   WEGE3                 WEG            Motores, compressores e outros   
1    ITSA              Itaúsa                                    Outros   
2   VIVT3          Telefônica                          Telecomunicações   
3   BBSE3       BB Seguridade                               Seguradoras   
4   B3SA3                  B3             Serviços financeiros diversos   
5   CPFE3                CPFL                          Energia elétrica   
6   EQTL3  Equatorial Energia                          Energia elétrica   
7   CXSE3    Caixa Seguridade                               Seguradoras   
8   RENT3            Localiza                         Aluguel de carros   
9   PSSA3        Porto Seguro                               Seguradoras   
10  NEOE3          Neoenergia                          Energia elétrica   
11  TOTS3               Totvs                      Programas e serviços   
12  RADL3       Raia Drog

In [1]:
import yfinance as yf
import pandas as pd

def get_historico(ticker):
    df = yf.download(f"{ticker}.SA", period="5y", interval="1d", progress=False)
    if df.empty or 'Close' not in df.columns:
        raise ValueError(f"Nenhum dado de histórico para {ticker}")
    df.reset_index(inplace=True)
    df = df.rename(columns={"Close": "close"})
    return df

def get_fundamental(ticker):
    stock = yf.Ticker(f"{ticker}.SA")
    info = stock.info
    pe = info.get("trailingPE")
    dy = info.get("dividendYield", 0)
    crescimento = info.get("earningsGrowth", 0)

    if pe is None:
        raise ValueError(f"P/L não encontrado para {ticker}")
    
    return {
        "peRatio": pe,
        "dividendYield": dy or 0,
        "netIncomeGrowth": crescimento or 0
    }

def calcula_indicador(df_hist, fn):
    pl_hist = fn.get("peRatio")
    dy_hist = fn.get("dividendYield")
    pl_atual = fn.get("peRatio")
    dy_atual = fn.get("dividendYield", 0)
    crescimento_lucro = fn.get("netIncomeGrowth", 0)

    if df_hist.empty or 'close' not in df_hist.columns:
        raise ValueError("Histórico vazio ou sem coluna 'close'")

    preco = df_hist['close'].iloc[-1]
    suporte = df_hist['close'].tail(6).min()
    media200 = df_hist['close'].rolling(window=200).mean().iloc[-1] if len(df_hist) >= 200 else preco
    score_tec = 1 if preco > media200 else -1

    indicador = 0
    if pl_hist and pl_atual:
        indicador += 30*((pl_hist - pl_atual)/pl_hist)
    if dy_hist:
        indicador += 25*((dy_atual - dy_hist)/dy_hist)
    indicador += 20 * crescimento_lucro
    indicador -= 15*((preco - suporte)/suporte)
    indicador += 10 * score_tec

    return round(indicador, 2), {
        "pl_hist": pl_hist, "pl_atual": pl_atual,
        "dy_hist": dy_hist, "dy_atual": dy_atual,
        "cres_lucro": crescimento_lucro,
        "preco": preco, "suporte": suporte,
        "score_tec": score_tec
    }


In [1]:
import requests
import pandas as pd
import yfinance as yf

API_BASE = "https://brapi.dev/api"
token='9vTikJ1xXMAMYKcLaCQuxM'
def get_historico(ticker):
    url = f"{API_BASE}/quote/{ticker}?range=3mo&interval=1d&token={token}"
    r = requests.get(url)
    r.raise_for_status()
    return pd.DataFrame(r.json().get("results", []))

# def get_fundamental(ticker):
#     stock = yf.Ticker(f"{ticker}.SA")
#     info = stock.info

#     return {
#         "peRatio": float(info.get("trailingPE") or 0),
#         "dividendYield": float(info.get("dividendYield") or 0),
#         "netIncomeGrowth": float(info.get("earningsGrowth") or 0)
#     }

def get_fundamental(ticker):
    stock = yf.Ticker(f"{ticker}.SA")
    info = stock.info
    pe = info.get("trailingPE")
    dy = info.get("dividendYield", 0)
    crescimento = info.get("earningsGrowth", 0)

    if pe is None:
        raise ValueError(f"P/L não encontrado para {ticker}")
    
    return {
        "peRatio": pe,
        "dividendYield": dy or 0,
        "netIncomeGrowth": crescimento or 0
    }


def calcula_indicador(df_hist, fn):
    try:
        pl_hist = float(fn.get("peRatio") or 0)
        dy_hist = float(fn.get("dividendYield") or 0)
        pl_atual = float(fn.get("peRatio") or 0)
        dy_atual = float(fn.get("dividendYield") or 0)
        crescimento_lucro = float(fn.get("netIncomeGrowth") or 0)

        if df_hist.empty or 'close' not in df_hist.columns:
            raise ValueError("Histórico vazio ou sem coluna 'close'")

        preco = df_hist['close'].iloc[-1]
        suporte = df_hist['close'].tail(6).min()
        media200 = df_hist['close'].rolling(window=200).mean().iloc[-1] if len(df_hist) >= 200 else preco
        score_tec = 1 if preco > media200 else -1

        indicador = 0
        if pl_hist > 0 and pl_atual > 0:
            indicador += 30 * ((pl_hist - pl_atual) / pl_hist)
        if dy_hist > 0:
            indicador += 25 * ((dy_atual - dy_hist) / dy_hist)
        indicador += 20 * crescimento_lucro
        if suporte > 0:
            indicador -= 15 * ((preco - suporte) / suporte)
        indicador += 10 * score_tec

        return round(indicador, 2), {
            "pl_hist": pl_hist, "pl_atual": pl_atual,
            "dy_hist": dy_hist, "dy_atual": dy_atual,
            "cres_lucro": crescimento_lucro,
            "preco": preco, "suporte": suporte,
            "score_tec": score_tec
        }
    except Exception as e:
        raise ValueError(f"Erro no cálculo do indicador: {e}")



In [2]:
get_historico("WEGE3")

,currency,marketCap,shortName,longName,regularMarketChange,regularMarketChangePercent,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,...,fiftyTwoWeekHigh,symbol,logourl,usedInterval,usedRange,historicalDataPrice,validRanges,validIntervals,priceEarnings,earningsPerShare
0,BRL,155797919599,WEG ON ED NM,WEG S.A.,0.84,2.201,2025-08-07T20:07:39.000Z,39,39.63,38.16 - 39.63,...,59.83,WEGE3,https://icons.brapi.dev/icons/WEGE3.svg,1d,3mo,"[{'date': 1746795600, 'open': 42.5, 'high': 42...","[1d, 2d, 5d, 7d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10...","[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1...",24.872365,1.527994


In [6]:
tickers = get_acoes_sem_prejuizo()
print("Ações sem prejuízo encontradas:", tickers)

Ações sem prejuízo encontradas: ['WEGE3', 'ITSA', 'VIVT3', 'BBSE3', 'B3SA3', 'CPFE3', 'EQTL3', 'CXSE3', 'RENT3', 'PSSA3', 'NEOE3', 'TOTS3', 'RADL3', 'ENGI', 'UGPA3', 'LREN3', 'GMAT3', 'ISAE', 'TFCO4', 'MULT3', 'ALOS3', 'TAEE', 'SAPR', 'POMO', 'ALUP', 'CURY3', 'SLCE3', 'PORT3', 'FLRY3', 'AZZA3', 'ODPV3', 'WHRL', 'FRAS3', 'SMTO3', 'DXCO3', 'GRND3', 'INTB3', 'VAMO3', 'LEVE3', 'EZTC3', 'MDIA3', 'FESA4', 'BLAU3', 'LOGG3', 'SHUL4', 'EUCA', 'CAML3', 'PNVL3', 'WIZC3', 'ALLD3', 'CSUD3', 'MELK3', 'CGRA', 'MTSA4']


In [26]:
import requests
import pandas as pd

API_BASE = "https://brapi.dev/api"
token='9vTikJ1xXMAMYKcLaCQuxM'
ticker = get_acoes_sem_prejuizo() 
url = f"{API_BASE}/quote/{ticker[0]}?token={token}"
response = requests.get(url)
data = response.json()
print(data)

{'results': [{'currency': 'BRL', 'marketCap': 155797919599, 'shortName': 'WEG         ON  ED  NM', 'longName': 'WEG S.A.', 'regularMarketChange': 0.84, 'regularMarketChangePercent': 2.2009999999999996, 'regularMarketTime': '2025-08-07T20:07:39.000Z', 'regularMarketPrice': 39, 'regularMarketDayHigh': 39.63, 'regularMarketDayRange': '38.16 - 39.63', 'regularMarketDayLow': 38.16, 'regularMarketVolume': 14200600, 'regularMarketPreviousClose': 38.16, 'regularMarketOpen': 37.13, 'fiftyTwoWeekRange': '35.75 - 59.83', 'fiftyTwoWeekLow': 35.75, 'fiftyTwoWeekHigh': 59.83, 'symbol': 'WEGE3', 'logourl': 'https://icons.brapi.dev/icons/WEGE3.svg', 'priceEarnings': 24.87236549286556, 'earningsPerShare': 1.5279943}], 'requestedAt': '2025-08-07T21:20:57.922Z', 'took': '0ms'}


In [7]:
def main():
    tickers = get_acoes_sem_prejuizo()
    print("Ações sem prejuízo encontradas:", tickers[1])

    resultados = {}
    for ticker in tickers:
        try:
            df = get_historico(ticker)
            fn = get_fundamental(ticker)
            ind, info = calcula_indicador(df, fn)
            resultados[ticker] = {"indicador":ind, **info}
        except Exception as e:
            print(f"Erro com {ticker}: {e}")

    for t, res in resultados.items():
        print(f"\n{t}: Indicador = {res['indicador']}")
        if res['indicador'] >= 70:
            print(" → Situação A de Compra")
        elif res['indicador'] <= -70:
            print(" → Situação A de Venda")
        elif res['indicador'] >= 30:
            print(" → Compra com cautela")
        elif res['indicador'] <= -30:
            print(" → Venda parcial ou observar")
        else:
            print(" → Zona neutra")


In [ ]:
main()

Ações sem prejuízo encontradas: ITSA
Erro com WEGE3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com ITSA: 404 Client Error: Not Found for url: https://brapi.dev/api/quote/ITSA?range=3mo&interval=1d&token=9vTikJ1xXMAMYKcLaCQuxM
Erro com VIVT3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com BBSE3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com B3SA3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com CPFE3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com EQTL3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com CXSE3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com RENT3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com PSSA3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com NEOE3: Erro no cálculo do indicador: Histórico vazio ou sem co

Exception ignored from cffi callback <function buffer_callback at 0x0000014637FEC360>:
Traceback (most recent call last):
  File "c:\Users\uriel\Desktop\bot_cripto\.venv\Lib\site-packages\curl_cffi\curl.py", line 100, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


Erro com ODPV3: Failed to perform, curl: (23) Failure writing output to destination, passed 13 returned 0. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
Erro com WHRL: 404 Client Error: Not Found for url: https://brapi.dev/api/quote/WHRL?range=3mo&interval=1d&token=9vTikJ1xXMAMYKcLaCQuxM
Erro com FRAS3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com SMTO3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com DXCO3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com GRND3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com INTB3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com VAMO3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com LEVE3: P/L não encontrado para LEVE3
Erro com EZTC3: Erro no cálculo do indicador: Histórico vazio ou sem coluna 'close'
Erro com MDIA3: Erro no cálculo do indicador: His